# 데이터셋 준비하기

---

## 0.사전 준비 사항
아래 사항을 먼저 준비 하셔야 합니다.

- huggingface Acess Key 준비 하기 : [User access tokens](https://huggingface.co/docs/hub/en/security-tokens)
- Llama-3-8B 모델 엑세스 권한 얻기: [meta-llama/Meta-Llama-3-8B](https://huggingface.co/meta-llama/Meta-Llama-3-8B)

## 1. 환경 셋업

### Hugging Face Token 입력
- [중요] HF Key 가 노출이 안되도록 조심하세요.

In [2]:
import os

def set_hf_key_env_vars(hf_key_name, key_val):
    os.environ[hf_key_name] = key_val

def get_hf_key_env_vars(hf_key_name):
    HF_key_value = os.environ.get(hf_key_name)

    return HF_key_value


is_sagemaker_notebook = True
if is_sagemaker_notebook:
    hf_key_name = "HF_KEY"
    key_val = "<Type Your HF Key>"
    set_hf_key_env_vars(hf_key_name, key_val)
    HF_TOKEN = get_hf_key_env_vars(hf_key_name)
else: # VS Code
    from dotenv import load_dotenv
    HF_TOKEN = os.getenv('HF_TOKEN')


# Log in to HF
!huggingface-cli login --token {HF_TOKEN}


/bin/sh: -c: line 0: syntax error near unexpected token `newline'
/bin/sh: -c: line 0: `huggingface-cli login --token <Type Your HF Key>'


#### 환경 변수 확인 ( HF 의 디폴트 경로 변경 함)

In [3]:
import os 
#os.environ["TRANSFORMERS_CACHE"] = "/home/ec2-user/SageMaker/.cache"
os.environ["HF_DATASETS_CACHE"] = "/home/ec2-user/SageMaker/.cache"
os.environ["HF_HOME"] = "/home/ec2-user/SageMaker/.cache"

#print("TRANSFORMERS_CACHE: ", os.getenv('TRANSFORMERS_CACHE'))
print("HF_DATASETS_CACHE: ", os.getenv('HF_DATASETS_CACHE'))
print("HF_HOME: ", os.getenv('HF_HOME'))

HF_DATASETS_CACHE:  /home/ec2-user/SageMaker/.cache
HF_HOME:  /home/ec2-user/SageMaker/.cache


In [4]:
import torch
import time
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

from datasets import Dataset, load_dataset
from datasets import load_dataset
from transformers import pipeline, set_seed
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import warnings
warnings.filterwarnings("ignore")

## 2. 데이터 셋 준비
### 데이터 셋 다운로드

In [5]:
huggingface_dataset_name = "daekeun-ml/naver-news-summarization-ko"

# dataset = load_dataset(huggingface_dataset_name, "3.0.0")
dataset = load_dataset(huggingface_dataset_name)
dataset

DatasetDict({
    train: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 22194
    })
    validation: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 2466
    })
    test: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 2740
    })
})

In [6]:
dataset["train"][0]

{'date': '2022-07-03 17:14:37',
 'category': 'economy',
 'press': 'YTN ',
 'title': '추경호 중기 수출지원 총력 무역금융 40조 확대',
 'document': '앵커 정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 했습니다. 특히 수출 중소기업의 물류난 해소를 위해 무역금융 규모를 40조 원 이상 확대하고 물류비 지원과 임시선박 투입 등을 추진하기로 했습니다. 류환홍 기자가 보도합니다. 기자 수출은 최고의 실적을 보였지만 수입액이 급증하면서 올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록했습니다. 정부가 수출확대에 총력을 기울이기로 한 것은 원자재 가격 상승 등 대외 리스크가 가중되는 상황에서 수출 증가세 지속이야말로 한국경제의 회복을 위한 열쇠라고 본 것입니다. 추경호 경제부총리 겸 기획재정부 장관 정부는 우리 경제의 성장엔진인 수출이 높은 증가세를 지속할 수 있도록 총력을 다하겠습니다. 우선 물류 부담 증가 원자재 가격 상승 등 가중되고 있는 대외 리스크에 대해 적극 대응하겠습니다. 특히 중소기업과 중견기업 수출 지원을 위해 무역금융 규모를 연초 목표보다 40조 원 늘린 301조 원까지 확대하고 물류비 부담을 줄이기 위한 대책도 마련했습니다. 이창양 산업통상자원부 장관 국제 해상운임이 안정될 때까지 월 4척 이상의 임시선박을 지속 투입하는 한편 중소기업 전용 선복 적재 용량 도 현재보다 주당 50TEU 늘려 공급하겠습니다. 하반기에 우리 기업들의 수출 기회를 늘리기 위해 2 500여 개 수출기업을 대상으로 해외 전시회 참가를 지원하는 등 마케팅 지원도 벌이기로 했습니다. 정부는 또 이달 중으로 반도체를 비롯한 첨단 산업 육성 전략을 마련해 수출 증가세를 뒷받침하고 에너지 소비를 줄이기 위한 효율화 방안을 마련해 무역수지 개선에 나서기로 했습니다. YTN 류환홍입니다.',
 'link': 'https://n.news.naver.com/mne

## 3. 데이터셋 변형

### Chat Message 형태 템플릿 정의

In [7]:
import json

def format_instruction(system_prompt: str, article: str, summary: str):
    message = [
            {
                'content': system_prompt,
                'role': 'system'
            },
            {
                'content': f'Please summarize the goals for journalist in this text:\n\n{article}',
                'role': 'user'
            },
            {
                'content': f'{summary}',
                'role': 'assistant'
            }
        ]
    
    return message # json.dumps(message, indent=2) # json.dumps(message, ensure_ascii=False, indent=2)


# 사용 예시
# system_prompt = "You are an AI assistant specialized in news articles. Your role is to provide accurate summaries and insights. Please analyze the given text and provide concise, informative summaries that highlight the key goals and findings."
# article = "Within three days, the intertwined cup nest of grasses was complete, featuring a canopy of overhanging grasses to conceal it. And decades later, it served as Rinkert's portal to the past inside the California Academy of Sciences. Information gleaned from such nests, woven long ago from species in plant communities called transitional habitat, could help restore the shoreline in the future. Transitional habitat has nearly disappeared from the San Francisco Bay, and scientists need a clearer picture of its original species composition—which was never properly documented. With that insight, conservation research groups like the San Francisco Bay Bird Observatory can help guide best practices when restoring the native habitat that has long served as critical refuge for imperiled birds and animals as adjacent marshes flood more with rising sea levels. \"We can't ask restoration ecologists to plant nonnative species or to just take their best guess and throw things out there,\" says Rinkert."
# summary = "Scientists are studying nests hoping to learn about transitional habitats that could help restore the shoreline of San Francisco Bay."

# print(format_instruction(system_prompt, article, summary))

### Chat Message 형태로 변환

In [8]:
# Add system message to each conversation
columns_to_remove = list(dataset["train"].features)
columns_to_remove

['date', 'category', 'press', 'title', 'document', 'link', 'summary']

In [9]:

def generate_instruction_dataset(data_point):
    system_prompt = "You are an AI assistant specialized in news articles.Your role is to provide accurate summaries and insights in Korean. Please analyze the given text and provide concise, informative summaries that highlight the key goals and findings."

    return {
        "messages": format_instruction(system_prompt, data_point["document"],data_point["summary"])
    }

def process_dataset(data: Dataset):
    return (
        data.shuffle(seed=42)
        .map(generate_instruction_dataset).remove_columns(columns_to_remove)
    )    

#### 전체 데이터 셋에서 일부 데티터 추출 (짧은 실습을 위해서)

In [10]:
train_num_debug = 10
validation_num_debug = 10
test_num_debug = 10


In [11]:
def create_message_dataset(dataset, num_train,num_val, num_test, verbose=False):
    ## APPLYING PREPROCESSING ON WHOLE DATASET

    train_dataset = process_dataset(dataset["train"].select(range(num_train)))
    validation_dataset = process_dataset(dataset["validation"].select(range(num_val)))
    test_dataset= process_dataset(dataset["test"].select(range(num_test)))
    
    if verbose:
        print(train_dataset)
        print(test_dataset)
        print(validation_dataset)

    return train_dataset,test_dataset,validation_dataset    

train_dataset,test_dataset,validation_dataset = create_message_dataset(dataset=dataset, 
                                                num_train=train_num_debug,
                                                num_val=validation_num_debug, 
                                                num_test=test_num_debug, verbose=True)    

Dataset({
    features: ['messages'],
    num_rows: 10
})
Dataset({
    features: ['messages'],
    num_rows: 10
})
Dataset({
    features: ['messages'],
    num_rows: 10
})


In [12]:
train_dataset[0]

{'messages': [{'content': 'You are an AI assistant specialized in news articles.Your role is to provide accurate summaries and insights in Korean. Please analyze the given text and provide concise, informative summaries that highlight the key goals and findings.',
   'role': 'system'},
  {'content': 'Please summarize the goals for journalist in this text:\n\n4일부터 석달간 증권사 신용융자담보비율 유지의무 면제 금융당국이 코스피지수가 장중 2300 아래까지 떨어지자 주식시장 변동성을 완화하는 조치를 시행하기로 했다. 금융위원회는 1일 주식시장 마감 후 김소영 부위원장 주재로 증권 유관기관과 금융시장합동점검회의를 열고 변동성 완화조치를 시행하기로 했다고 밝혔다. 이에 따라 다음 주식시장 개장일인 오는 4일부터 9월30일까지 3개월간 증권사의 신용융자담보비율 유지의무가 면제된다. 신용융자담보비율 유지의무란 증권사가 일명 ‘빚투’ 빚내서 투자 자금인 신용융자를 시행할 때 담보를 140% 이상 확보하고 내규에서 정한 담보비율을 유지할 것을 요구하는 규제다. 유지의무 면제는 증권사의 신용융자 담보주식에 대한 과도한 반대매매를 억제하기 위한 조치다. 금융당국은 코로나19 사태가 본격화한 2020년 3월에도 이 같은 조치를 6개월간 시행한 바 있다. 7일부터 10월6일까지는 상장기업의 하루 자기주식 매수 주문 수량 한도 제한도 완화된다. 신탁취득 주식도 발행주식총수의 1% 이내에서 신탁재산 총액 범위 내로 한시적으로 확대된다. 이와 함께 금융위는 금융감독원과 한국거래소 합동으로 공매도 특별점검을 실시해 공매도 현황과 시장교란 가능성 등을 살펴보기로 했다. 현재 공매도는 지난해 5월 이후 제한적

#### 전체 데이터 셋 저장 (성능 측정을 위해서)

In [13]:
full_train_num = len(dataset["train"])
full_validation_num = len(dataset["validation"])
full_test_num = len(dataset["test"])

# full_train_num = 1000
# full_validation_num = 1000
# full_test_num = 1000

print("train_num_samples: ", full_train_num)
print("validation_num_samples: ", full_validation_num)
print("test_num_samples: ", full_test_num)

full_train_dataset,full_test_dataset,full_validation_dataset = create_message_dataset(dataset=dataset, 
                                                                num_train=full_train_num,
                                                                num_val=full_validation_num, 
                                                                num_test=full_test_num, verbose=True)    

train_num_samples:  22194
validation_num_samples:  2466
test_num_samples:  2740
Dataset({
    features: ['messages'],
    num_rows: 22194
})
Dataset({
    features: ['messages'],
    num_rows: 2740
})
Dataset({
    features: ['messages'],
    num_rows: 2466
})


## 4. 데이터 셋을 JSON 으로 저장

In [14]:
import os

def create_dataset_json_file(huggingface_dataset_name,train_dataset, validation_dataset, test_dataset, is_full, verbose=True ):
    dataset_name = huggingface_dataset_name.split("/")[1]
    data_folder = os.path.join("../data/",dataset_name)
    os.makedirs(data_folder, exist_ok=True)

    if is_full:
        train_data_json = os.path.join(data_folder,"full_train", "train_dataset.json")
        validation_data_json = os.path.join(data_folder,"full_validation", "validation_dataset.json")
        test_data_json = os.path.join(data_folder, "full_test", "test_dataset.json")

    else:
        train_data_json = os.path.join(data_folder,"train", "train_dataset.json")
        validation_data_json = os.path.join(data_folder,"validation", "validation_dataset.json")
        test_data_json = os.path.join(data_folder, "test", "test_dataset.json")

    # save datasets to disk 
    train_dataset.to_json(train_data_json, orient="records", force_ascii=False)
    validation_dataset.to_json(validation_data_json, orient="records", force_ascii=False)
    test_dataset.to_json(test_data_json, orient="records", force_ascii=False)        

    if verbose:
        print(train_dataset)
        print(f"{train_data_json} is saved")
        print(f"{validation_data_json} is saved")
        print(f"{test_data_json} is saved")                

    return data_folder, train_data_json, validation_data_json, test_data_json



In [15]:
# Store debug dataset
data_folder, train_data_json, validation_data_json, test_data_json = create_dataset_json_file(huggingface_dataset_name=huggingface_dataset_name,
                                                                    train_dataset=train_dataset, 
                                                                    validation_dataset=validation_dataset, 
                                                                    test_dataset=test_dataset,
                                                                    is_full=False )        

# Store full dataset
data_folder, full_train_data_json, full_validation_data_json, full_test_data_json = create_dataset_json_file(huggingface_dataset_name=huggingface_dataset_name,
                                                                    train_dataset=full_train_dataset, 
                                                                    validation_dataset=full_validation_dataset, 
                                                                    test_dataset=full_test_dataset,
                                                                    is_full=True )        


Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Dataset({
    features: ['messages'],
    num_rows: 10
})
../data/naver-news-summarization-ko/train/train_dataset.json is saved
../data/naver-news-summarization-ko/validation/validation_dataset.json is saved
../data/naver-news-summarization-ko/test/test_dataset.json is saved


Creating json from Arrow format:   0%|          | 0/23 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Dataset({
    features: ['messages'],
    num_rows: 22194
})
../data/naver-news-summarization-ko/full_train/train_dataset.json is saved
../data/naver-news-summarization-ko/full_validation/validation_dataset.json is saved
../data/naver-news-summarization-ko/full_test/test_dataset.json is saved


### 다음 노트북에서 사용하기 위해 변수 저장

In [16]:
%store data_folder
%store train_data_json 
%store validation_data_json 
%store test_data_json 
%store full_train_data_json 
%store full_validation_data_json 
%store full_test_data_json

Stored 'data_folder' (str)
Stored 'train_data_json' (str)
Stored 'validation_data_json' (str)
Stored 'test_data_json' (str)
Stored 'full_train_data_json' (str)
Stored 'full_validation_data_json' (str)
Stored 'full_test_data_json' (str)


## 5. Option: 데이터 셋을 ChatTemplate 형태로 바꾸기

### Chat Template 정의

In [17]:
LLAMA_3_CHAT_TEMPLATE = (
    "{% for message in messages %}"
        "{% if message['role'] == 'system' %}"
            "{{ message['content'] }}"
        "{% elif message['role'] == 'user' %}"
            "{{ '\n\nHuman: ' + message['content'] +  eos_token }}"
        "{% elif message['role'] == 'assistant' %}"
            "{{ '\n\nAssistant: '  + message['content'] +  eos_token  }}"
        "{% endif %}"
    "{% endfor %}"
    "{% if add_generation_prompt %}"
    "{{ '\n\nAssistant: ' }}"
    "{% endif %}"
)
LLAMA_3_CHAT_TEMPLATE

"{% for message in messages %}{% if message['role'] == 'system' %}{{ message['content'] }}{% elif message['role'] == 'user' %}{{ '\n\nHuman: ' + message['content'] +  eos_token }}{% elif message['role'] == 'assistant' %}{{ '\n\nAssistant: '  + message['content'] +  eos_token  }}{% endif %}{% endfor %}{% if add_generation_prompt %}{{ '\n\nAssistant: ' }}{% endif %}"

### Chat Template 으로 변형하기

In [18]:
# Tokenizer        
#tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B", use_fast=True)
tokenizer = AutoTokenizer.from_pretrained("MLP-KTLim/llama-3-Korean-Bllossom-8B", use_fast=True)


tokenizer.pad_token = tokenizer.eos_token
tokenizer.chat_template = LLAMA_3_CHAT_TEMPLATE

# template dataset
def template_dataset(examples):
    return{"text":  tokenizer.apply_chat_template(examples["messages"], tokenize=False)}

sample_train_dataset = train_dataset.map(template_dataset, remove_columns=["messages"])
# sample_test_dataset = test_dataset.map(template_dataset, remove_columns=["messages"])
# sample_validation_dataset = validation_dataset.map(template_dataset, remove_columns=["messages"])    

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

### 변형된 Chat Message 형태 예시 보기

In [19]:
# print random sample
import random

for index in random.sample(range(len(sample_train_dataset)), 1):
    print("index: ", index)
    # index = 5343
    print(sample_train_dataset[index]["text"])

index:  7
You are an AI assistant specialized in news articles.Your role is to provide accurate summaries and insights in Korean. Please analyze the given text and provide concise, informative summaries that highlight the key goals and findings.

Human: Please summarize the goals for journalist in this text:

정부가 허준이 미국 프린스턴대 교수 겸 한국 고등과학원 수학부 석학교수의 필즈상 수상을 축하했습니다. 과학기술정보통신부는 대수기하학의 토대가 더욱 확장되도록 새 지평을 연 공로를 인정받아 한국계 최초로 필즈상 수상의 쾌거를 이룩했다 며 축하했습니다. 또 허 교수의 필즈상 수상은 올해 국제수학연맹에서 한국의 수학 국가등급을 최고등급으로 상향한 것에 이은 기쁜 소식 이라며 허 교수의 행보와 한국 수학계의 지속적 발전 역시 기대된다고 덧붙였습니다.<|eot_id|>

Assistant: 정부가 허준이 미국 프린스턴대 교수 겸 한국 고등과학원 수학부 석학교수의 필즈상 수상을 축하했고 과학기술정보통신부는 대수기하학의 토대가 더욱 확장되도록 새 지평을 연 공로를 연 공로를 연 공로를 연 공로를 연 공로를 연 공로를 연 공로를 연 허준이 미국 프린스턴대 교수 겸 한국 고등과학원 수학부 석학교수의 필즈상 수상을 축하했다.<|eot_id|>
